<a href="https://colab.research.google.com/github/Iamkoss/cis9650_flight_data/blob/main/fligh_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis of Flight Data from the Buffalo, NY Airport

### Author: Anna Bold, Kossi Gamli, Ran Xin Jiang, Christina Pratas
### Date : December 8th, 2025

## Executive Summary

## Table of Contents

1. Introduction
2. Problem Statement / Research Question
3. Data Description
4. Setup and Environment
5. Data Loading
6. Data Preparation
7. Model Planning
8. Model Building / Analysis
9. Discussion & Interpretation
10. Conclusion
11. References
12. Appendix

## Introduction
This projects examines flight records from Buffalo Niagara International Airport (BUF) for the entirety of January 2025. The following 30 JSON files contain detailed information regarding flight schedules, delays, airports, and timezones, etc. The dataset shows how flights operated throughout the month, including patterns in delays, cancellations, airline performance, and schedule consistency. By organizing this raw data into a structured format, it highlights these trends and provides a clearer picture of BUF's activity during this period.

## Problem Statement / Research Question

The research question is finding out what overlying issues the Buffalo Niagara International Airport may be having with the daily arriving and departing flights in January 2025. There are several different factors to look at, including potential problematic airlines, large discrepancies between scheduled departure/arrival times and actual departure/arrival times, and outlying flight cancellations.


As Buffalo in January is subject to inclement weather, research will also look into if any singular days are outliers in regards to flight delays and cancellations due to weather.

## Data Description

## Setup and Environment

In [ ]:
!pip install google-cloud-storage

In [77]:
!git clone https://github.com/Iamkoss/cis9650_flight_data.git

fatal: destination path 'cis9650_flight_data' already exists and is not an empty directory.


In [78]:
from google.cloud import storage
from io import BytesIO
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import json
import os
import glob
from google.cloud import storage
from io import BytesIO

## Data Loading

In [87]:
# Using glob library to get all the data the /content/cis9650_flight_data/BUF with .json extension
files = sorted(glob.glob("/content/cis9650_flight_data/BUF*.json"))

# empty variable to store all the files
dfs = []

# loop for iterating though the files
for file in files:
    # Message to show that file is laoding
    print(f'Loading:, {file}')
    # reading the files as a DataFrame
    df = pd.read_json(file)
    # Adding all to dfs
    dfs.append(df)
    #
    all_dfs = pd.concat(dfs)

print(all_dfs)


Loading:, /content/cis9650_flight_data/BUF_1.json
Loading:, /content/cis9650_flight_data/BUF_10.json
Loading:, /content/cis9650_flight_data/BUF_11.json
Loading:, /content/cis9650_flight_data/BUF_12.json
Loading:, /content/cis9650_flight_data/BUF_13.json
Loading:, /content/cis9650_flight_data/BUF_14.json
Loading:, /content/cis9650_flight_data/BUF_15.json
Loading:, /content/cis9650_flight_data/BUF_16.json
Loading:, /content/cis9650_flight_data/BUF_17.json
Loading:, /content/cis9650_flight_data/BUF_18.json
Loading:, /content/cis9650_flight_data/BUF_19.json
Loading:, /content/cis9650_flight_data/BUF_2.json
Loading:, /content/cis9650_flight_data/BUF_20.json
Loading:, /content/cis9650_flight_data/BUF_21.json
Loading:, /content/cis9650_flight_data/BUF_22.json
Loading:, /content/cis9650_flight_data/BUF_23.json
Loading:, /content/cis9650_flight_data/BUF_24.json
Loading:, /content/cis9650_flight_data/BUF_25.json
Loading:, /content/cis9650_flight_data/BUF_26.json
Loading:, /content/cis9650_flight

In [88]:
flight_record = df.to_dict(orient="records")

df_all = pd.json_normalize(flight_record)
display(df_all.head())
print(df_all.shape)
print(len(df_all))

,flight_date,flight_status,aircraft,live,departure.airport,departure.timezone,departure.iata,departure.icao,departure.terminal,departure.gate,...,flight.codeshared.airline_iata,flight.codeshared.airline_icao,flight.codeshared.flight_number,flight.codeshared.flight_iata,flight.codeshared.flight_icao,flight.codeshared,aircraft.registration,aircraft.iata,aircraft.icao,aircraft.icao24
0,2025-01-28,landed,NaN,NaN,Buffalo Niagara International,America/New_York,BUF,KBUF,None,4,...,aa,aal,5312,aa5312,aal5312,NaN,NaN,NaN,NaN,NaN
1,2025-01-28,landed,NaN,NaN,Buffalo Niagara International,America/New_York,BUF,KBUF,None,4,...,NaN,NaN,NaN,NaN,NaN,NaN,N516AE,CRJ7,CRJ7,A677BE
2,2025-01-28,landed,NaN,NaN,Buffalo Niagara International,America/New_York,BUF,KBUF,None,5,...,NaN,NaN,NaN,NaN,NaN,NaN,N129AA,A321,A321,A07522
3,2025-01-28,landed,NaN,NaN,Buffalo Niagara International,America/New_York,BUF,KBUF,None,2A,...,NaN,NaN,NaN,NaN,NaN,NaN,N652RS,E145,E145,A895B0
4,2025-01-28,landed,NaN,NaN,Buffalo Niagara International,America/New_York,BUF,KBUF,None,8,...,b6,jbu,141,b6141,jbu141,NaN,NaN,NaN,NaN,NaN


(173, 46)
173


In [89]:
print(files)
print(len(files))


['/content/cis9650_flight_data/BUF_1.json', '/content/cis9650_flight_data/BUF_10.json', '/content/cis9650_flight_data/BUF_11.json', '/content/cis9650_flight_data/BUF_12.json', '/content/cis9650_flight_data/BUF_13.json', '/content/cis9650_flight_data/BUF_14.json', '/content/cis9650_flight_data/BUF_15.json', '/content/cis9650_flight_data/BUF_16.json', '/content/cis9650_flight_data/BUF_17.json', '/content/cis9650_flight_data/BUF_18.json', '/content/cis9650_flight_data/BUF_19.json', '/content/cis9650_flight_data/BUF_2.json', '/content/cis9650_flight_data/BUF_20.json', '/content/cis9650_flight_data/BUF_21.json', '/content/cis9650_flight_data/BUF_22.json', '/content/cis9650_flight_data/BUF_23.json', '/content/cis9650_flight_data/BUF_24.json', '/content/cis9650_flight_data/BUF_25.json', '/content/cis9650_flight_data/BUF_26.json', '/content/cis9650_flight_data/BUF_27.json', '/content/cis9650_flight_data/BUF_28.json', '/content/cis9650_flight_data/BUF_29.json', '/content/cis9650_flight_data/BUF

In [90]:
# work

# Convert DataFrame to list of dictionaries
flight_record = dfs.to_dict(orient="records")

# Check first record
print(flight_record[0])

# Show how many flights we have
print("Total flights:", len(flight_record))


AttributeError: 'list' object has no attribute 'to_dict'

In [91]:
#NEEDS WORKS, DONT KNOW IF WE WILL USE THIS, IT CAME FROM THE EXAMPLE
def gcs_file_to_dataframe(bucket_name: str, blob_path: str):

      # Download file content as bytes


    # Detect file extension

        # Convert bytes → Python object


        # If JSON is a list → NORMAL DataFrame


        # If JSON is a dict → flatten into rows

SyntaxError: incomplete input (ipython-input-2798657046.py, line 14)

In [ ]:
#NEEDS WORKS, BUT IT WILL ALLOW US TO CONVERT THE DICTIONARY INTO COLUMNS, AND THEN WE SHOULD BE ABLE TO SELECT THE COLUMNS WE WANT TO WORK ON.

df_expanded = df.explode("details", ignore_index=True)

# 2. Convert the dict into columns
detail_cols = df_expanded["rows"].apply(pd.Series)

# 3. Merge back and drop the dict column
df_flat = pd.concat([df_expanded.drop(columns=["rows"]), detail_cols], axis=1)

df_flat

KeyError: 'details'

## Data Preparation

In [ ]:
display(df)

,flight_date,flight_status,departure,arrival,airline,flight,aircraft,live
0,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...",{'airport': 'Ronald Reagan Washington National...,"{'name': 'AlphaSky', 'iata': 'AS', 'icao': 'JAG'}","{'number': '6276', 'iata': 'AS6276', 'icao': '...",None,NaN
1,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...",{'airport': 'Ronald Reagan Washington National...,"{'name': 'American Airlines', 'iata': 'AA', 'i...","{'number': '5312', 'iata': 'AA5312', 'icao': '...","{'registration': 'N516AE', 'iata': 'CRJ7', 'ic...",NaN
2,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'Charlotte Douglas', 'timezone': '...","{'name': 'American Airlines', 'iata': 'AA', 'i...","{'number': '1204', 'iata': 'AA1204', 'icao': '...","{'registration': 'N129AA', 'iata': 'A321', 'ic...",NaN
3,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'Philadelphia International', 'tim...","{'name': 'American Airlines', 'iata': 'AA', 'i...","{'number': '5769', 'iata': 'AA5769', 'icao': '...","{'registration': 'N652RS', 'iata': 'E145', 'ic...",NaN
4,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'John F Kennedy International', 't...","{'name': 'British Airways', 'iata': 'BA', 'ica...","{'number': '8304', 'iata': 'BA8304', 'icao': '...",None,NaN
...,...,...,...,...,...,...,...,...
168,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'John F Kennedy International', 't...","{'name': 'Qatar Airways', 'iata': 'QR', 'icao'...","{'number': '3936', 'iata': 'QR3936', 'icao': '...",None,NaN
169,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'Philadelphia International', 'tim...","{'name': 'UPS Airlines', 'iata': '5X', 'icao':...","{'number': '1131', 'iata': '5X1131', 'icao': '...","{'registration': 'N473UP', 'iata': 'B752', 'ic...",NaN
170,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'Fort Lauderdale Executive', 'time...","{'name': 'Private owner', 'iata': None, 'icao'...","{'number': '148', 'iata': None, 'icao': None, ...",None,NaN
171,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...",{'airport': 'Raleigh-durham International Airp...,"{'name': 'NetJets Aviation', 'iata': '1I', 'ic...","{'number': '958', 'iata': '1I958', 'icao': 'EJ...",None,NaN


In [ ]:
def clean_data(df):
    # Standardize column names
    df.columns = df.columns.str.lower().str.replace(' ', '_')

    # Remove duplicate rows
    df.drop_duplicates(inplace=True)

    return df
all_dfs = clean_data(all_dfs)

TypeError: unhashable type: 'dict'

In [ ]:
display(df)

,flight_date,flight_status,departure,arrival,airline,flight,aircraft,live
0,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...",{'airport': 'Ronald Reagan Washington National...,"{'name': 'AlphaSky', 'iata': 'AS', 'icao': 'JAG'}","{'number': '6276', 'iata': 'AS6276', 'icao': '...",None,NaN
1,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...",{'airport': 'Ronald Reagan Washington National...,"{'name': 'American Airlines', 'iata': 'AA', 'i...","{'number': '5312', 'iata': 'AA5312', 'icao': '...","{'registration': 'N516AE', 'iata': 'CRJ7', 'ic...",NaN
2,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'Charlotte Douglas', 'timezone': '...","{'name': 'American Airlines', 'iata': 'AA', 'i...","{'number': '1204', 'iata': 'AA1204', 'icao': '...","{'registration': 'N129AA', 'iata': 'A321', 'ic...",NaN
3,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'Philadelphia International', 'tim...","{'name': 'American Airlines', 'iata': 'AA', 'i...","{'number': '5769', 'iata': 'AA5769', 'icao': '...","{'registration': 'N652RS', 'iata': 'E145', 'ic...",NaN
4,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'John F Kennedy International', 't...","{'name': 'British Airways', 'iata': 'BA', 'ica...","{'number': '8304', 'iata': 'BA8304', 'icao': '...",None,NaN
...,...,...,...,...,...,...,...,...
168,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'John F Kennedy International', 't...","{'name': 'Qatar Airways', 'iata': 'QR', 'icao'...","{'number': '3936', 'iata': 'QR3936', 'icao': '...",None,NaN
169,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'Philadelphia International', 'tim...","{'name': 'UPS Airlines', 'iata': '5X', 'icao':...","{'number': '1131', 'iata': '5X1131', 'icao': '...","{'registration': 'N473UP', 'iata': 'B752', 'ic...",NaN
170,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'Fort Lauderdale Executive', 'time...","{'name': 'Private owner', 'iata': None, 'icao'...","{'number': '148', 'iata': None, 'icao': None, ...",None,NaN
171,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...",{'airport': 'Raleigh-durham International Airp...,"{'name': 'NetJets Aviation', 'iata': '1I', 'ic...","{'number': '958', 'iata': '1I958', 'icao': 'EJ...",None,NaN


## Model Planning

## Model building / Analysis

## Discussion and Results

## Conclusion

## References

1. Barber, David. Bayesian Reasoning and Machine Learning. Cambridge University Press, 2012.
2. Aste, Tomaso, Paola Cerchiello, and Roberta Scaramozzino. "Information-Theoretic Causality Detection between Financial and Sentiment Data."Entropy, vol. 24, no. 6, 2022, pp. 1–18. DOI:10.3390/e24060774.
3. Metz, Cade. "Microsoft Puts OpenAI’s Sam Altman in Charge of New Advanced AI Research Team."
The New York Times, 20 Nov. 2023, www.nytimes.com/2023/11/20/technology/openai-microsoft-altman.html

## Appendix